In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/spam_ham_india.csv")
df.head()

,Msg,Label
0,CONGRATULATIONS! FREE 2GB data is yours! Claim...,spam
1,Hi! Thank you for being with Vi-India's FASTES...,spam
2,"As part of Cyber Swachhta Pakhwada, CERT-In Go...",spam
3,I will try to manage took tablets,ham
4,Study from Home with Vi!! Watch Kite Victers C...,spam


In [ ]:
input_path = "/content/SMSSpamCollection"
output_path = "sms_spam_english.csv"
df = pd.read_csv(input_path, sep="\t", header=None, names=["label", "text"])
df = df.dropna(subset=["text"])
df["label"] = df["label"].str.strip().str.lower()

In [ ]:
print(df.head(10))
print(f"\nTotal rows: {len(df)}")
print(df['label'].value_counts())

In [ ]:
df.to_csv(output_path, index=False, encoding="utf-8")

In [ ]:
import pandas as pd
import re

# ====== INPUT FILES ======
uci_path = "sms_spam_english.csv"     # from UCI dataset
india_path = "spam_ham_india.csv"     # your Indian dataset
output_path = "merged_clean_sms.csv"

print("🔹 Loading datasets...")
uci_df = pd.read_csv(uci_path)
india_df = pd.read_csv(india_path)

# ====== RENAME INDIA DATASET COLUMNS ======
india_df = india_df.rename(columns={"Msg": "text", "Label": "label"})

# ====== KEEP CONSISTENT COLUMNS ======
uci_df = uci_df[["label", "text"]]
india_df = india_df[["label", "text"]]

# ====== NORMALIZE LABELS ======
uci_df["label"] = uci_df["label"].str.strip().str.lower()
india_df["label"] = india_df["label"].str.strip().str.lower()

# ====== COMBINE ======
combined_df = pd.concat([uci_df, india_df], ignore_index=True)

# ====== CLEAN TEXT ======
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "URL", text)        # mask URLs
    text = re.sub(r"\S+@\S+", "EMAIL", text)             # mask emails
    text = re.sub(r"\d{10,}", "PHONE", text)             # mask phone numbers
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)           # remove special chars
    text = re.sub(r"\s+", " ", text).strip()             # normalize spaces
    return text

print("🔹 Cleaning text...")
combined_df["text"] = combined_df["text"].apply(clean_text)

# ====== DROP DUPLICATES & EMPTY ROWS ======
combined_df = combined_df.drop_duplicates(subset="text").dropna(subset=["text"])

# ====== LABEL CHECK ======
print("\nLabel distribution:")
print(combined_df["label"].value_counts())

# ====== SAVE ======
combined_df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Clean merged dataset saved as: {output_path}")
print(f"Total samples: {len(combined_df)}")


🔹 Loading datasets...
🔹 Cleaning text...

Label distribution:
label
ham     5761
spam    1329
Name: count, dtype: int64

✅ Clean merged dataset saved as: merged_clean_sms.csv
Total samples: 7090


In [ ]:
df2 = pd.read_csv("/content/merged_clean_sms.csv")
df2.head()

,label,text
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [ ]:
df3 = pd.read_csv("/content/hinglish_sms_dataset.csv")
df3.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/hinglish_sms_dataset.csv'

In [ ]:
!pip install -q transformers datasets scikit-learn torch matplotlib

# ✅ 2. Import libraries
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")
# ✅ 4. Load dataset (upload merged_clean_sms.csv in Colab first)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("merged_clean_sms.csv")
print("Dataset loaded:", df.shape)
print(df.head())

Running on: cuda


KeyboardInterrupt: 

In [ ]:
# ✅ 5. Encode labels
df["label"] = df["label"].str.lower().map({"ham": 0, "spam": 1})
print(df["label"].value_counts())


In [ ]:
# ✅ Clean text data to ensure all entries are strings
def clean_texts(series):
    # convert non-string to string, drop NaN
    series = series.dropna()
    series = series.astype(str)
    # strip spaces and weird characters
    series = series.str.strip()
    return series

train_texts = clean_texts(pd.Series(train_texts)).tolist()
test_texts = clean_texts(pd.Series(test_texts)).tolist()

print(f"✅ Cleaned train set size: {len(train_texts)} | test set size: {len(test_texts)}")
print(train_texts[:5])


In [ ]:
# ✅ 6. Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42,
)



In [ ]:
# ✅ 7. Tokenization
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels
})
test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "labels": test_labels
})

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast

# --- Reload dataset cleanly ---
df = pd.read_csv("/content/merged_clean_sms.csv")

# Normalize column names just in case
df.columns = df.columns.str.lower()

# Convert all label text to string categories ('ham', 'spam')
df['label'] = df['label'].astype(str).str.strip().str.lower()

# --- Sanitize text column ---
def clean_texts(series):
    cleaned = []
    for x in series:
        if isinstance(x, float) and pd.isna(x):
            continue
        if not isinstance(x, str):
            x = str(x)
        x = x.strip()
        if x != "":
            cleaned.append(x)
    return cleaned

texts = clean_texts(df["text"])
labels = [1 if lbl == "spam" else 0 for lbl in df["label"][:len(texts)]]

print(f"✅ Cleaned dataset size: {len(texts)} samples")

# --- Train/test split ---
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# --- Tokenization ---
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)

print("🔥 Tokenization successful.")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW # Import AdamW from torch.optim
from transformers.optimization import get_linear_schedule_with_warmup # Correct import for the scheduler
from tqdm import tqdm

# ✅ 1. Dataset class
class SMSDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        # Add print statements to check lengths when dataset is initialized
        print(f"SMSDataset initialized - encodings lengths: {[len(v) for v in encodings.values()]}, labels length: {len(labels)}")

    def __getitem__(self, idx):
        # Add print statements to debug indexing
        #print(f"Accessing index: {idx}")
        #print(f"Encodings lengths in __getitem__: {[len(v) for v in self.encodings.values()]}")
        #print(f"Labels length in __getitem__: {len(self.labels)}")

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SMSDataset(train_encodings, train_labels)
test_dataset = SMSDataset(test_encodings, test_labels)

# ✅ 2. Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

# ✅ 3. Training parameters
batch_size = 8
epochs = 4
learning_rate = 2e-5

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Calculate total training steps for the scheduler
# total_steps = len(train_loader) * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# ✅ 4. Training loop
# model.train()
# for epoch in range(epochs):
#     print(f"\n🚀 Epoch {epoch+1}/{epochs}")
#     total_loss = 0
#     for batch in tqdm(train_loader):
#         batch = {k: v.to(device) for k, v in batch.items()}
#         optimizer.zero_grad()
#         outputs = model(**batch)
#         loss = outputs.loss
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
#         scheduler.step() # Step the scheduler
#     avg_loss = total_loss / len(train_loader)
#     print(f"Average training loss: {avg_loss:.4f}")

# ✅ 5. Save model
# model.save_pretrained("bert_phishing_english")
# print("\n✅ Model training complete and saved as 'bert_phishing_english'")

In [ ]:
# ✅ 6. Evaluation
model.eval() # Set the model to evaluation mode
predictions = []
true_labels = []

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size) # Use the same batch size as training

with torch.no_grad(): # Disable gradient calculation for evaluation
    # Iterate through the test_loader instead of test_dataset directly
    for batch in tqdm(test_loader):
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().tolist())
        true_labels.extend(batch["labels"].cpu().tolist())

# Convert predictions and true_labels to numpy arrays for scikit-learn metrics
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["ham", "spam"]))

# Plot confusion matrix
cm = confusion_matrix(true_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["ham", "spam"])
disp.plot()
plt.title("Confusion Matrix")
plt.show()

print("\n✅ Evaluation complete.")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from transformers import BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW # Import AdamW from torch.optim
from transformers.optimization import get_linear_schedule_with_warmup # Correct import for the scheduler
from tqdm import tqdm


model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/bert_phishing_english")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # Move the model to the device


# Switch to evaluation mode
model.eval()

predictions, true_labels = [], []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=16):
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch["labels"].cpu().numpy())

# Convert to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Metrics
print("✅ Evaluation Results:")
print(f"Accuracy:  {accuracy_score(true_labels, predictions):.4f}")
print(f"Precision: {precision_score(true_labels, predictions):.4f}")
print(f"Recall:    {recall_score(true_labels, predictions):.4f}")
print(f"F1-score:  {f1_score(true_labels, predictions):.4f}")
print("\nDetailed Report:")
print(classification_report(true_labels, predictions, target_names=["ham", "spam"]))

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification

# --- Reload dataset ---
df = pd.read_csv("/content/merged_clean_sms.csv")
df.columns = df.columns.str.lower()
df['label'] = df['label'].astype(str).str.strip().str.lower()

# Clean text
texts = [str(t).strip() for t in df['text'] if str(t).strip() != ""]
labels = [1 if l == 'spam' else 0 for l in df['label'][:len(texts)]]

# Split again (same random seed)
from sklearn.model_selection import train_test_split
_, test_texts, _, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Tokenize
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)

# Recreate dataset class
class SMSDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

test_dataset = SMSDataset(test_encodings, test_labels)

# --- Reload model ---
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/bert_phishing_english")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader
import torch

# --- Evaluation ---
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=16):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        preds.extend(predictions.cpu().numpy())
        true_labels.extend(batch["labels"].cpu().numpy())

# --- Metrics ---
acc = accuracy_score(true_labels, preds)
prec = precision_score(true_labels, preds)
rec = recall_score(true_labels, preds)
f1 = f1_score(true_labels, preds)
cm = confusion_matrix(true_labels, preds)

print(f"📊 Accuracy: {acc:.4f}")
print(f"🎯 Precision: {prec:.4f}")
print(f"🔁 Recall: {rec:.4f}")
print(f"🏁 F1 Score: {f1:.4f}")
print("\nConfusion Matrix:\n", cm)


In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification
import torch

model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/bert_phishing_english")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model.eval()

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
    return "spam" if pred == 1 else "ham"

print(predict("Hi, please review the attached document about your pending appraisal and sign it before Friday."))
print(predict("You won $1,000! Click here to claim your prize!!"))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

ham
spam


In [ ]:
model.save_pretrained("bert_phishing_english")
tokenizer.save_pretrained("bert_phishing_english")


('bert_phishing_english/tokenizer_config.json',
 'bert_phishing_english/special_tokens_map.json',
 'bert_phishing_english/vocab.txt',
 'bert_phishing_english/added_tokens.json',
 'bert_phishing_english/tokenizer.json')